In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

In [2]:
import os, sys, time

In [3]:
import ROOT

Welcome to JupyROOT 6.10/09


In [4]:
from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

In [5]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

In [6]:
class MFaJaT(Module):
    def __init__(self):
        self.writeHistFile=True
        self.counter = 0
    def beginJob(self,histFile=None,histDirName=None):
        Module.beginJob(self,histFile,histDirName)
        
        self.h_lumiblock = ROOT.TH1I('h_lumiblock', ':LumiBlock:Events',   100, 0, 1000)
        self.addObject(self.h_lumiblock)
        self.h_run = ROOT.TH1I('h_run', ':Run:Events', 10, 273400, 277000)
        self.addObject(self.h_run)
    #def endJob(self):
    #    pass #Cannot do pass in endJob if we write histograms... they don't get written! So do no override Module 
    #def beginFile(self, inputFile, outputFile):
    #def beginFile(self):
        #pass
    #def endFile(self, inputFile, outputFile):
    #def endFile(self):
        #pass
    def analyze(self, event):
        """process event, return True (go to next module) or False (fail, go to next event)"""
        
        modulator = 1000 # how often to print an event when we do self.counter % modulator
        
        ###########################################
        ###### Basic Attributes of the Event ######
        ###########################################
        run = getattr(event, "run")
        lumi = getattr(event, "luminosityBlock")
        evt = getattr(event, "event")

        self.h_lumiblock.Fill(lumi)
        self.h_run.Fill(run)
        if self.counter % modulator == 0:
            print("Run: {0:>8d} LuminosityBlock: {2:>8d} Event: {1:>8d}".format(run, evt, lumi)) #swizzling because lazy
        
        ################################
        ###### High Level Trigger ######
        ################################
        HLT = Object(event, "HLT") #Grab all the HLT triggers 'together'
        passTrig=["IsoMu20", "IsoMu24"] #Create a list of valid triggers to check, dropping "HLT_"
        if self.counter % modulator == 0:
            for trig in passTrig:
                print("\tHLT_" + str(trig) + " Trigger: " + str(getattr(HLT, trig)) )
            
            
        ###############################
        ######### MET Filters #########
        ###############################
        Filters = Object(event, "Flag") #Grab all the MET Filters in a group
        passFilter=["HBHENoiseFilter", "HBHENoiseIsoFilter", "EcalDeadCellTriggerPrimitiveFilter", 
                    "globalSuperTightHalo2016Filter", "goodVertices", "METFilters", "noBadMuons"] #All the filters commonly used 
        if self.counter % modulator == 0:
            for fltr in passFilter:
                print("\t\tFlag_" + str(fltr) + " Filter: " + str(getattr(Filters, fltr)))
        
        
        self.counter += 1
        return True 

In [7]:
#preselection="nMuon > 0 && nElectron > 0"
preselection=None
filePrefix = "root://cms-xrd-global.cern.ch/"
inputList = open("../data/Run2016/Run2016B_SM", "r")
files=[]
for line in inputList:
    files.append(filePrefix + str(line))
#for file in files:
#    print(file)
onefile = [files[0]] #Experiment with just one file for now...
#onefile = " root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root"
print(onefile)

[u'root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root\n']


In [8]:
p=PostProcessor(".",onefile,cut=preselection,branchsel=None,modules=[MFaJaT()],friend=False,postfix=None, 
                jsonInput=None,noOut=True,justcount=False,provenance=False,
                haddFileName=None,fwkJobReport=False,histFileName="histOut.root",
                histDirName="plots", outputbranchsel=None)
#jsonInput: json file in dictionary format {"RunNumberInt": [[lumilow,lumihigh],[lumi2low,lumi2high]], "RunNumber2Int":[[low,high]]}
#files: list ["fileone.root","filetwo.root"] of inputs, even if one file must be a list!
#branchsel: if non-None, selection of branches to not even activate/load into memory
#outputbranchsel: if non-None, selection of branches to still include in output (see noOut)
#noOut: If True, no output of skimmed data is written. If False, will write full data file fitting outputbranch selections and postfix concattenated to name
#postfix: string added to inputfile name to indicate this module processed it!
#histFileName: name of any output file for histograms created in your class, as above with runs and lumis
#histDirName: name INSIDE the "histFileName.root" file's directory structure!
#friend: Not tested
#provenance: not tested
#haddFileName: Not properly tested (tied together with fwkJobReport)
#fwkJobReport: only relevant with multiple files being added together? Not properly tested

In [9]:
p.run()

Pre-select 11647 entries out of 11647 

/afs/cern.ch/work/n/nmangane/scratch1/CMSSW_9_4_9/python/PhysicsTools/NanoAODTools/postprocessing/framework/treeReaderArrayTools.py:69: RuntimeWarning: creating executor for unknown type "ULong64_t*"
  ret = _vr.Get()[0]



Run:   273410 LuminosityBlock:       50 Event: 84373408
	HLT_IsoMu20 Trigger: False
	HLT_IsoMu24 Trigger: False
		Flag_HBHENoiseFilter Filter: True
		Flag_HBHENoiseIsoFilter Filter: True
		Flag_EcalDeadCellTriggerPrimitiveFilter Filter: True
		Flag_globalSuperTightHalo2016Filter Filter: True
		Flag_goodVertices Filter: True
		Flag_METFilters Filter: True
		Flag_noBadMuons Filter: True
Run:   273410 LuminosityBlock:       50 Event: 84732209
	HLT_IsoMu20 Trigger: False
	HLT_IsoMu24 Trigger: False
		Flag_HBHENoiseFilter Filter: True
		Flag_HBHENoiseIsoFilter Filter: True
		Flag_EcalDeadCellTriggerPrimitiveFilter Filter: True
		Flag_globalSuperTightHalo2016Filter Filter: True
		Flag_goodVertices Filter: True
		Flag_METFilters Filter: True
		Flag_noBadMuons Filter: True
Run:   273410 LuminosityBlock:       51 Event: 87173441
	HLT_IsoMu20 Trigger: True
	HLT_IsoMu24 Trigger: True
		Flag_HBHENoiseFilter Filter: True
		Flag_HBHENoiseIsoFilter Filter: True
		Flag_EcalDeadCellTriggerPrimitiveFil

In [10]:
p2=PostProcessor(".",onefile,cut=preselection,branchsel=None,modules=[MFaJaT()],friend=False,postfix="_MFaJaT", 
                jsonInput="../data/json/testjson.json",noOut=False,justcount=False,provenance=False,
                haddFileName=None,fwkJobReport=False,histFileName="histOut.root",
                histDirName="plots", outputbranchsel=None)
#Add json file, add postfix name, change noOut to False

In [11]:
p2.run()

Will write selected trees to .
Pre-select 7331 entries out of 11647 
Run:   273410 LuminosityBlock:       50 Event: 84373408
	HLT_IsoMu20 Trigger: False
	HLT_IsoMu24 Trigger: False
		Flag_HBHENoiseFilter Filter: True
		Flag_HBHENoiseIsoFilter Filter: True
		Flag_EcalDeadCellTriggerPrimitiveFilter Filter: True
		Flag_globalSuperTightHalo2016Filter Filter: True
		Flag_goodVertices Filter: True
		Flag_METFilters Filter: True
		Flag_noBadMuons Filter: True
Run:   273410 LuminosityBlock:       50 Event: 84732209
	HLT_IsoMu20 Trigger: False
	HLT_IsoMu24 Trigger: False
		Flag_HBHENoiseFilter Filter: True
		Flag_HBHENoiseIsoFilter Filter: True
		Flag_EcalDeadCellTriggerPrimitiveFilter Filter: True
		Flag_globalSuperTightHalo2016Filter Filter: True
		Flag_goodVertices Filter: True
		Flag_METFilters Filter: True
		Flag_noBadMuons Filter: True
Run:   273410 LuminosityBlock:       53 Event: 89579473
	HLT_IsoMu20 Trigger: False
	HLT_IsoMu24 Trigger: False
		Flag_HBHENoiseFilter Filter: True
		Flag_